Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы (его тут нет, потому что он слишком большой для Git'a). Из этого архива вам нужен лишь файл checkins.dat. Он лежит в архиве checkins.dat.tar.gz

In [1]:
import re
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift

In [2]:
def write_to_csv(data, file):
    """
    Метод пишет данные в csv-файл. Входные данные подаются в виде списка. В файле будет строка вида:
    text1,text2,text3,text4,text5\n
    :param data: список
    :param file: файл, куда надо писать
    :return: None
    """
    for i in range(0, 6):
        if i != 5:
            file.write(data[i] + ",")
        else:
            file.write(data[i] + "\n")
    return


def create_csv_from_txt(file_txt, file_csv):
    line = file_txt.readline()
    header = re.findall("[\w]+", line)
    write_to_csv(header, file_csv)

    file_txt.readline()

    idx = 0
    for line in file_txt:
        data = re.split("\|", line)
        if len(data) < 6:
            break
        else:
            for i in range(0, 5):
                data[i] = re.findall("[\s]*([\w\.\-]*)", data[i])[0]
            data[5] = re.findall("[\s]*([\w\-: ]*)", data[5])[0]
            idx += 1
            if data[3] == "" or data[4] == "":
                continue
            write_to_csv(data, file_csv)
    return

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

In [3]:
with open("checkins.dat", "r") as file_txt:
    with open("data.csv", "w") as file_csv:
        create_csv_from_txt(file_txt, file_csv)

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [4]:
data = pd.read_csv("data.csv", header=0)
len(data)

396634

In [ ]:
data.head()

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [6]:
X = data.values[0:100000, 3:5]
model = MeanShift(bandwidth=0.1)
model.fit(X)

labels = model.labels_
cluster_centers = model.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [10]:
label_counts = {}
for label in labels:
    if label not in label_counts.keys():
        label_counts[label] = 1
    else:
        label_counts[label] += 1
        
count = 0
for key in label_counts.keys():
    if label_counts[key] > 15:
        count += 1
print("Число кластеров, где больше 15-ти элементов:", count)

Число кластеров, где больше 15-ти элементов: 592


In [11]:
selected_clusters = np.ndarray(shape=(count, 2))

i = 0
j = 0
while i < len(cluster_centers):
    if label_counts[i] > 15:
        selected_clusters[j] = cluster_centers[i]
        j += 1
    i += 1

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

In [12]:
offices = np.ndarray(shape=(6,2))
offices[0] = np.array([33.751277, -118.188740])
offices[1] = np.array([25.867736, -80.324116])
offices[2] = np.array([51.503016, -0.075479])
offices[3] = np.array([52.378894, 4.885084])
offices[4] = np.array([39.366487, 117.036146])
offices[5] = np.array([-33.868457, 151.205134])

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.



In [13]:
def compute_distance(point1, point2):
    return ((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)**0.5

In [14]:
min_distance = 0
answer_index = 0
i = 0
while i < len(selected_clusters):
    distances = [compute_distance(coordinates_of_office, selected_clusters[i]) for coordinates_of_office in offices]
    if min_distance == 0:
        min_distance = min(distances)
        answer_index = i
    else:
        if min_distance > min(distances):
            min_distance = min(distances)
            answer_index = i
    i += 1

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса.

In [15]:
selected_clusters[answer_index]

array([ -33.86063043,  151.20477593])

 Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [16]:
with open("1.txt", "w") as file:
    file.write(str(selected_clusters[0]) + " " + str(selected_clusters[1]))